# Software Project

In [16]:
import numpy as np
import pandas as pd
import re
import json
import os
from konlpy.tag import Okt
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

DATA_IN_PATH = 'C:/jupyter_project/Estimation _Analysis_with_Tensorflow_and_Natural_Language_Processing/data/DATA_IN/'
DIR_PATH = 'C:/jupyter_project/Estimation _Analysis_with_Tensorflow_and_Natural_Language_Processing/data/DATA_IN/nsmc/'

train_data = pd.read_csv(DATA_IN_PATH+'갓지웅.txt', header=0, delimiter='\t', quoting=3)
test_data = pd.read_csv(DATA_IN_PATH+'test_wong.txt', header=0, delimiter='\t', quoting=3)

In [11]:
def preprocessing(check, okt, remove_stopwords = False, stop_words = []):
    result = []
    tmp = re.compile('[^가-힣]+')
    res = tmp.sub('', check)
    
    res = okt.morphs(res, stem=True)
    
    for w in res:
        if w not in stop_words:
            result.append(w)
    return result

In [12]:
def classify(data_arr):
    sim = [['과제', '레포트', '플', '조별', '발표', '독후감', '프로젝트'],
           ['난이도', '시험', '중간고사', '기말고사', '중간', '기', '말', '변별', '범위', '셤', '문제', '퀴즈'],
           ['출석', '탈주', '출결', '출첵'],
           ['수업', '강의', '진도', '과목', '전공', '교양', '내용', '설명', '퀄리티', '학기'],
           ['인성', '교수', '인격']]
    temp = [[0]*0 for i in range(len(sim))]
    key = [[0]*0 for i in range(len(sim))]
    tmp = 0
    cnt = 0
    for i in data_arr:
        last = -1
        now = -1
        for j in i:
            for k in sim:
                for l in k:
                    if j == l:
                        cnt+=1
                        now = i.index(j, last+1)
                        if last != -1:
                            for m in sim:
                                for n in m:
                                    if n == i[last]:
                                        tmp = sim.index(m)
                            if tmp != sim.index(k):
                                storage = tmp
                            else:
                                storage = sim.index(k)
                            while last < now-1:
                                temp[storage].append(i[last+1])
                                last+=1
                    last = now
        if cnt != 0:
            for m in sim:
                for n in m:
                    if n == i[now]:
                        tmp = sim.index(m)
            while(last < len(i)-1):
                temp[tmp].append(i[last+1])
                last+=1
        key[0].append(temp[0])
        key[1].append(temp[1])
        key[2].append(temp[2])
        key[3].append(temp[3])
        key[4].append(temp[4])
        cnt = 0
        temp = [[0]*0 for i in range(len(sim))]        
    return key

In [13]:
def trans_label(attr, label): #라벨을 6개의 속성에 따라 분리
    tmp = []
    cnt = 0
    for i in label:
        if attr == 0:
            tmp.append(int(i/10000))
        elif attr == 1:
            tmp.append(int(i/1000%10))
        elif attr == 2:
            tmp.append(int(i/100%10))
        elif attr == 3:
            tmp.append(int(i%100/10))
        elif attr == 4:
            tmp.append(int(i%10))
    return tmp
    
def cal_len(arr):
    max = 0
    for i in arr:
        if max < len(i):
            max = len(i)
    return max

In [14]:
def call_Tok(key, raw_data, path): #raw_data에서 label값 추출
    str = ['과제', '시험', '출석', '수업', '인성']
    temp = np.array(raw_data['label'])
    last = -1
    for i in key:
        tmp = key.index(i, last+1)
        label_arr = trans_label(tmp, temp)
        
        #label이 2에 해당하는 데이터와 라벨 없애기
        prev = -1
        del_label_index = []
        for j in label_arr:
            if j == 2:
                now = label_arr.index(j, prev+1)
                del_label_index.append(now)
                prev = now
        for j in del_label_index:
            del label_arr[j-del_label_index.index(j)]
            del i[j-del_label_index.index(j)]

        DATA = path+"data_"+str[tmp]+".npy"
        LABEL = path+"label_"+str[tmp]+".npy"
        CONFIGS = path+str[tmp]+"_configs.json"
        
        Tok(i, label_arr, DATA, LABEL, CONFIGS)
        last = tmp
        
def Tok(key, label_arr, DATA, LABEL, CONFIGS):
    #패딩처리 : 레코드나 블록의 맨 나중에공백이나 의미가 없는 기호를 부가하여 고정 길이로 만드는 것"""
    tok = Tokenizer()
    tok.fit_on_texts(key) #입력에 맞게 내부의 word_index를 만듬
    seq = tok.texts_to_sequences(key)
    maxlen = cal_len(key)
    data = pad_sequences(seq, maxlen, padding='post')
    
    word_vocab = tok.word_index
    data_configs = {}
    data_configs['vocab'] = word_vocab
    data_configs['vocab_size'] = len(word_vocab)+1

    np.save(open(DIR_PATH+DATA, 'wb'), data)
    np.save(open(DIR_PATH+LABEL, 'wb'), label_arr)
    json.dump(data_configs, open(DIR_PATH+CONFIGS, 'w'), ensure_ascii=False)

In [17]:
okt = Okt()
stop_words = set(['도','흠','은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한'])
train_res = []
test_res = []

for check in train_data ['document']:
    if type(check) == str:
        train_res.append(preprocessing(check, okt, remove_stopwords = True, stop_words = stop_words))
    else:
        train_res.append([])
        
for check in test_data ['document']:
    if type(check) == str:
        test_res.append(preprocessing(check, okt, remove_stopwords = True, stop_words = stop_words))
    else:
        test_res.append([])

In [18]:
train_key = classify(train_res)
test_key = classify(test_res)

call_Tok(train_key, train_data, 'train_')
call_Tok(test_key, test_data, 'test_')